In [1]:
#import SparkSession
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('random_forest').getOrCreate()

In [2]:
#read the dataset
df=spark.read.csv('affairs.csv',inferSchema=True,header=True)

In [3]:
#check the shape of the data 
print("Shape of dataset",(df.count(),len(df.columns)))

Shape of dataset (6366, 6)


In [4]:
#printSchema
df.printSchema()

root
 |-- rate_marriage: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- yrs_married: double (nullable = true)
 |-- children: double (nullable = true)
 |-- religious: integer (nullable = true)
 |-- affairs: integer (nullable = true)



In [5]:
df.show(10)

+-------------+----+-----------+--------+---------+-------+
|rate_marriage| age|yrs_married|children|religious|affairs|
+-------------+----+-----------+--------+---------+-------+
|            5|32.0|        6.0|     1.0|        3|      0|
|            4|22.0|        2.5|     0.0|        2|      0|
|            3|32.0|        9.0|     3.0|        3|      1|
|            3|27.0|       13.0|     3.0|        1|      1|
|            4|22.0|        2.5|     0.0|        1|      1|
|            4|37.0|       16.5|     4.0|        3|      1|
|            5|27.0|        9.0|     1.0|        1|      1|
|            4|27.0|        9.0|     0.0|        2|      1|
|            5|37.0|       23.0|     5.5|        2|      1|
|            5|37.0|       23.0|     5.5|        2|      1|
+-------------+----+-----------+--------+---------+-------+
only showing top 10 rows



In [6]:
#Exploratory Data Analysis
df.describe().select('summary','rate_marriage','age','yrs_married','children','religious').show()

+-------+------------------+------------------+-----------------+------------------+------------------+
|summary|     rate_marriage|               age|      yrs_married|          children|         religious|
+-------+------------------+------------------+-----------------+------------------+------------------+
|  count|              6366|              6366|             6366|              6366|              6366|
|   mean| 4.109644989004084|29.082862079798932| 9.00942507068803|1.3968740182218033|2.4261702796104303|
| stddev|0.9614295945655025| 6.847881883668817|7.280119972766412| 1.433470828560344|0.8783688402641785|
|    min|                 1|              17.5|              0.5|               0.0|                 1|
|    max|                 5|              42.0|             23.0|               5.5|                 4|
+-------+------------------+------------------+-----------------+------------------+------------------+



In [7]:
df.groupBy('affairs').count().show()

+-------+-----+
|affairs|count|
+-------+-----+
|      1| 2053|
|      0| 4313|
+-------+-----+



In [8]:
df.groupBy('rate_marriage').count().show()

+-------------+-----+
|rate_marriage|count|
+-------------+-----+
|            1|   99|
|            3|  993|
|            5| 2684|
|            4| 2242|
|            2|  348|
+-------------+-----+



In [10]:
df.groupBy('rate_marriage','affairs').count().orderBy('rate_marriage','affairs','count',ascending=True).show()

+-------------+-------+-----+
|rate_marriage|affairs|count|
+-------------+-------+-----+
|            1|      0|   25|
|            1|      1|   74|
|            2|      0|  127|
|            2|      1|  221|
|            3|      0|  446|
|            3|      1|  547|
|            4|      0| 1518|
|            4|      1|  724|
|            5|      0| 2197|
|            5|      1|  487|
+-------------+-------+-----+



In [11]:
df.groupBy('religious','affairs').count().orderBy('religious','affairs','count',ascending=True).show()

+---------+-------+-----+
|religious|affairs|count|
+---------+-------+-----+
|        1|      0|  613|
|        1|      1|  408|
|        2|      0| 1448|
|        2|      1|  819|
|        3|      0| 1715|
|        3|      1|  707|
|        4|      0|  537|
|        4|      1|  119|
+---------+-------+-----+



In [12]:
df.groupBy('children','affairs').count().orderBy('children','affairs','count',ascending=True).show()

+--------+-------+-----+
|children|affairs|count|
+--------+-------+-----+
|     0.0|      0| 1912|
|     0.0|      1|  502|
|     1.0|      0|  747|
|     1.0|      1|  412|
|     2.0|      0|  873|
|     2.0|      1|  608|
|     3.0|      0|  460|
|     3.0|      1|  321|
|     4.0|      0|  197|
|     4.0|      1|  131|
|     5.5|      0|  124|
|     5.5|      1|   79|
+--------+-------+-----+



In [13]:
df.groupBy('affairs').mean().show()

+-------+------------------+------------------+------------------+------------------+------------------+------------+
|affairs|avg(rate_marriage)|          avg(age)|  avg(yrs_married)|     avg(children)|    avg(religious)|avg(affairs)|
+-------+------------------+------------------+------------------+------------------+------------------+------------+
|      1|3.6473453482708234|30.537018996590355|11.152459814905017|1.7289332683877252| 2.261568436434486|         1.0|
|      0| 4.329700904242986| 28.39067934152562| 7.989334569904939|1.2388128912589844|2.5045212149316023|         0.0|
+-------+------------------+------------------+------------------+------------------+------------------+------------+



In [15]:
from pyspark.ml.feature import VectorAssembler

In [16]:
df_assembler = VectorAssembler(inputCols=['rate_marriage', 'age', 'yrs_married', 'children', 'religious'], outputCol="features")
df = df_assembler.transform(df)

In [17]:
df.printSchema()

root
 |-- rate_marriage: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- yrs_married: double (nullable = true)
 |-- children: double (nullable = true)
 |-- religious: integer (nullable = true)
 |-- affairs: integer (nullable = true)
 |-- features: vector (nullable = true)



In [18]:
df.select(['features','affairs']).show(10,False)

+-----------------------+-------+
|features               |affairs|
+-----------------------+-------+
|[5.0,32.0,6.0,1.0,3.0] |0      |
|[4.0,22.0,2.5,0.0,2.0] |0      |
|[3.0,32.0,9.0,3.0,3.0] |1      |
|[3.0,27.0,13.0,3.0,1.0]|1      |
|[4.0,22.0,2.5,0.0,1.0] |1      |
|[4.0,37.0,16.5,4.0,3.0]|1      |
|[5.0,27.0,9.0,1.0,1.0] |1      |
|[4.0,27.0,9.0,0.0,2.0] |1      |
|[5.0,37.0,23.0,5.5,2.0]|1      |
|[5.0,37.0,23.0,5.5,2.0]|1      |
+-----------------------+-------+
only showing top 10 rows



In [19]:
#select data for building model
model_df=df.select(['features','affairs'])

In [21]:
train_df,test_df = model_df.randomSplit([0.75,0.25])

In [23]:
print("Number of datapoints in train data:",train_df.count())

Number of datapoints in train data: 4779


In [24]:
train_df.groupBy('affairs').count().show()

+-------+-----+
|affairs|count|
+-------+-----+
|      1| 1530|
|      0| 3249|
+-------+-----+



In [25]:
test_df.groupBy('affairs').count().show()

+-------+-----+
|affairs|count|
+-------+-----+
|      1|  523|
|      0| 1064|
+-------+-----+



In [26]:
from pyspark.ml.classification import RandomForestClassifier

rf_classifier=RandomForestClassifier(labelCol='affairs',numTrees=50).fit(train_df)
rf_predictions=rf_classifier.transform(test_df)
rf_predictions.show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|affairs|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[1.0,22.0,2.5,1.0...|      1|[23.5704857021994...|[0.47140971404398...|       1.0|
|[1.0,22.0,2.5,1.0...|      1|[25.6534908991467...|[0.51306981798293...|       0.0|
|[1.0,27.0,2.5,0.0...|      1|[23.0780320955064...|[0.46156064191012...|       1.0|
|[1.0,27.0,6.0,1.0...|      0|[17.2419435853409...|[0.34483887170681...|       1.0|
|[1.0,27.0,6.0,1.0...|      0|[17.2419435853409...|[0.34483887170681...|       1.0|
|[1.0,27.0,6.0,2.0...|      1|[18.2117566340915...|[0.36423513268183...|       1.0|
|[1.0,32.0,13.0,2....|      1|[13.2770796537303...|[0.26554159307460...|       1.0|
|[1.0,32.0,13.0,3....|      1|[16.4962096644415...|[0.32992419328883...|       1.0|
|[1.0,32.0,16.5,2....|      1|[13.3944251517209...|[0.26788850303441...|    

- The third column (rawPrediction) represents the measure of confidence
for both possible outputs.

- The fourth column(probability) is that of conditional
probability of each class label,

- fifth column(prediction) is the prediction by the
random forest classifier

In [27]:
rf_predictions.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0| 1320|
|       1.0|  267|
+----------+-----+



In [28]:
rf_predictions.select(['probability','affairs','prediction']).show(10,False)

+----------------------------------------+-------+----------+
|probability                             |affairs|prediction|
+----------------------------------------+-------+----------+
|[0.4714097140439889,0.5285902859560111] |1      |1.0       |
|[0.5130698179829344,0.48693018201706556]|1      |0.0       |
|[0.4615606419101283,0.5384393580898716] |1      |1.0       |
|[0.34483887170681793,0.6551611282931821]|0      |1.0       |
|[0.34483887170681793,0.6551611282931821]|0      |1.0       |
|[0.3642351326818312,0.6357648673181688] |1      |1.0       |
|[0.2655415930746061,0.7344584069253939] |1      |1.0       |
|[0.3299241932888318,0.6700758067111681] |1      |1.0       |
|[0.2678885030344193,0.7321114969655806] |1      |1.0       |
|[0.3024031119782125,0.6975968880217875] |1      |1.0       |
+----------------------------------------+-------+----------+
only showing top 10 rows



In [29]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [30]:
rf_accuracy=MulticlassClassificationEvaluator(labelCol='affairs',metricName='accuracy').evaluate(rf_predictions)
print('The accuracy of RF on test data is {0:.2%}'.format(rf_accuracy))

The accuracy of RF on test data is 70.26%


In [31]:
rf_precision=MulticlassClassificationEvaluator(labelCol='affairs',metricName='weightedPrecision').evaluate(rf_predictions)
print('The precision rate on test data is {0:.2%}'.format(rf_precision))

The precision rate on test data is 68.18%


In [33]:
rf_auc=BinaryClassificationEvaluator(labelCol='affairs').evaluate(rf_predictions)
print(rf_auc)

0.7266672896390115


In [34]:
# Feature importance
rf_classifier.featureImportances


SparseVector(5, {0: 0.6039, 1: 0.0256, 2: 0.225, 3: 0.0793, 4: 0.0662})

In [35]:
df.schema["features"].metadata["ml_attr"]["attrs"]

{'numeric': [{'idx': 0, 'name': 'rate_marriage'},
  {'idx': 1, 'name': 'age'},
  {'idx': 2, 'name': 'yrs_married'},
  {'idx': 3, 'name': 'children'},
  {'idx': 4, 'name': 'religious'}]}

### Saving the Model